In [1]:
import matplotlib.pyplot as plt
import numpy as np

from sciopy import EIT_16_32_64_128, EitMeasurementSetup
import matplotlib.pyplot as plt

In [2]:
# create a 'sciospec' class which represents the sciospec EIT device
n_el = 16
sciospec = EIT_16_32_64_128(n_el)

In [3]:
# connect device via USB-HS port
sciospec.connect_device_HS()

In [4]:
# read system message buffer
sciospec.SystemMessageCallback()
# should be empty

No message inside the message buffer
message buffer:
 []
message length:	 0


In [5]:
# create a measurement setup
setup = EitMeasurementSetup(
    burst_count=5,
    n_el=n_el,
    exc_freq=125_000,
    framerate=3,
    amplitude=0.01,
    inj_skip=n_el // 2,
    gain=1,
    adc_range=1,
)

In [6]:
sciospec.SetMeasurementSetup(setup)

Command-Acknowledge: Command has been executed successfully
message buffer:
 ['0x18', '0x1', '0x83', '0x18']
message length:	 4
Command-Acknowledge: Command has been executed successfully
message buffer:
 ['0x18', '0x1', '0x83', '0x18']
message length:	 4
Command-Acknowledge: Command has been executed successfully
message buffer:
 ['0x18', '0x1', '0x83', '0x18']
message length:	 4
Command-Acknowledge: Command has been executed successfully
message buffer:
 ['0x18', '0x1', '0x83', '0x18']
message length:	 4


## Ender 5

In [7]:
import serial

from typing import Union
import time
from datetime import datetime
import numpy as np
import sys
import glob
from src.util import (
    createTrajectory,
)
import os
from glob import glob
# https://reprap.org/wiki/G-code#M17:_Enable.2FPower_all_stepper_motors


In [75]:
def command(ser, command) -> None:
    ser.write(str.encode(command))
    time.sleep(1)
    while True:
        line = ser.readline()
        print(line)

        if line == b"ok\n":
            break

# class
class EnderControl:
    def __init__(self,com_port, baudrate = 115200, timeout = 1, motion_speed = 1000):
        self.com_port = com_port
        self.baudrate = baudrate
        self.motion_speed = motion_speed # movement in mm/min
        self.position_timestamps = []  # Hier wird das Attribut position_timestamps korrekt initialisiert


    def connect(self):
        com = serial.Serial(
            port=self.com_port,
            baudrate=self.baudrate,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS,
        )
    
        print("Connection to", com.name, "is established.")
        self.com = com
        
    def init(self):
        """
        Initialise the axis
        """
        # x_y_home(ser)
        command(self.com, f"G28 X0 Y0 F{self.motion_speed}\r\n")
        command(self.com, f"G28 Z0 F{self.motion_speed}\r\n")
        self.x_y_center()
        self.turn_off_fan()
        print("X,Y axis are centered at X(180), Y(180)")
    
    def turn_off_fan(self):
        command(ser, "M106 S0\r\n")

    def x_y_center(self):
        command(self.com, f"G0 X180 Y180 F{self.motion_speed}\r\n")

    def move_to_absolute_x_y(self, P) -> None:
        command(
            self.com,
            f"G0 X{P[0]+180} Y{P[1]+180} F{self.motion_speed}\r\n",
        )
        pos = [P[0]+180, P[1]+180]
        return pos

    def read_temperature(self) -> float:
        """
        Read the bed temperature of the Ender 5.
    
        This function sends the M105 command, which requests the current temperature.
    
        Returns
        -------
        float
            The bed temperature value.
        """
    
        self.com.write(str.encode(f"M105\r\n"))
        time.sleep(1)
        line = self.com.readline()
        temp = float(str(line).split("B:")[1].split(" ")[0])
        
        return temp

    def get_timestamp(self) -> str:
        """
        Returns the current timestamp in the format YYYY-MM-DD HH:MM:SS
        """
        current_time = datetime.now()  # Get the current date and time
        formatted_timestamp = current_time.strftime("%Y-%m-%d %H:%M:%S")  # Format the timestamp
        return current_time, formatted_timestamp


In [76]:
printer = EnderControl(com_port="/dev/ttyUSB0")

In [77]:
printer.connect()

Connection to /dev/ttyUSB0 is established.


In [78]:
traj= "Kreis"  
r_path = 0.5  
Nsteps = 10 
material = "Glas"
r_anomaly = 20


**[EIT measurement](https://github.com/EITLabworks/sciopy/blob/USB_fs_rework/examples/example_notebook.ipynb)**

In [79]:
def generate_exp_data(traj, r_path, r_anomaly, Nsteps, material):

    center_pos = createTrajectory("Kreis", r_path*97, r_path_variations=None, bound=0, num_points=Nsteps)
    
    base_dataset_folder = "exp_data_set"
    os.makedirs(base_dataset_folder, exist_ok=True)
    base_name = "exp_data"
    counter = 1
    folder_name = os.path.join(base_dataset_folder, base_name)
    while os.path.exists(folder_name):
        folder_name = os.path.join(base_dataset_folder, f"{base_name}{counter}")
        counter += 1
    os.makedirs(folder_name) 
    
    lookup = {
            "folder": folder_name,
            "Trajectory": traj,
            "Nsteps": Nsteps,
            "r_anomaly": r_anomaly,
            "r_path": r_path,
            "material": material, 
        }
    
    lookup_file_path = os.path.join(base_dataset_folder, "exp_data_log.txt")
        
    with open(lookup_file_path, 'a') as f:
        f.write(f"Folder: {folder_name}\n")
        f.write(f"Trajectory: {traj}\n")
        f.write(f"Nsteps: {Nsteps}\n")
        f.write(f"r_anomaly: {r_anomaly}\n")
        f.write(f"r_path: {r_path}\n")
        f.write(f"material: {material}\n")
        f.write("-" * 40 + "\n")  
    
    for save_index, Ps in enumerate(center_pos):
        pos = printer.move_to_absolute_x_y(Ps)    #PROBLEM: er startet Messung bevor er die Position richtig anfahren konnte
        temp = printer.read_temperature()
        timestamp = printer.get_timestamp()
        data = sciospec.StartStopMeasurement(return_as="pot_mat")  
        file_path = os.path.join(folder_name, "sample_{0:06d}.npz".format(save_index))
        np.savez(file_path, position= pos, timestamp=timestamp, temperature = temp, v=data)

In [83]:
traj_type = "Kreis"
r_path = 0.50
r_anomaly = 20
Nsteps = 10
material = "idk yet"


generate_exp_data(traj_type, r_path, r_anomaly, Nsteps, material)

b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [82]:
data_set = "exp_data"
data_dirs = sorted(glob(f"exp_data_set/{data_set}/"))  
#absolut werte wichtig
for i, directory in enumerate(data_dirs):
    file_list = sorted(glob(f"{directory}*.npz"))  
    voltage_list = []
    position_list = []
    timestamp_list = []
    temp_list = []

    for file in file_list:
        tmp = np.load(file, allow_pickle=True)  
        voltage_list.append(tmp["v"])  
        position_list.append(tmp["position"])
        timestamp_list.append(tmp["timestamp"])
        temp_list.append(tmp["temperature"])

    voltage_array = np.array(voltage_list) 
    position_array = np.array(position_list)
    timestamp_array = np.array(timestamp_list)
    temperature_array = np.array(temp_list)


print(position_array)
print(timestamp_array)
print(temperature_array)
print(voltage_array)
        


[[228.5        180.        ]
 [217.15315549 211.17519907]
 [188.42193662 227.76317602]
 [155.75       222.00223208]
 [134.42490789 196.58797695]
 [134.42490789 163.41202305]
 [155.75       137.99776792]
 [188.42193662 132.23682398]
 [217.15315549 148.82480093]
 [228.5        180.        ]]
['2024-12-16 14:38:54' '2024-12-16 14:38:58' '2024-12-16 14:39:03'
 '2024-12-16 14:39:07' '2024-12-16 14:39:12' '2024-12-16 14:39:16'
 '2024-12-16 14:39:21' '2024-12-16 14:39:25' '2024-12-16 14:39:30'
 '2024-12-16 14:39:34']
[20.16 20.23 20.39 20.39 20.39 20.16 20.2  20.23 20.31 20.35]
[[[[-4.24122542e-01-1.19716489e+00j -8.90329406e-02-2.71832407e-01j
    -1.76607045e-06-1.76733215e-06j ... -1.76607045e-06-1.76733215e-06j
    -1.76607045e-06-1.76733215e-06j -1.76607045e-06-1.76733215e-06j]
   [-9.20216069e-02-2.60601133e-01j -4.19989198e-01-1.19866490e+00j
    -1.76607045e-06-1.76733215e-06j ... -1.76607045e-06-1.76733215e-06j
    -1.76607045e-06-1.76733215e-06j -1.76607045e-06-1.76733215e-06j]
   [

In [20]:
# nur das erste mal ausführen, um zu leveln
printer.init()

b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'X:360.00 Y:360.00 Z:10.00 E:0.00 Count X:28800 Y:28800 Z:8000\n'
b'ok\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'echo:busy: processing\n'
b'X:180.00 Y:180.00 Z:10.00 E:0.00 Count X:14400 Y:14400 Z:8000\n'
b'ok\n'
b'ok\n'


NameError: name 'ser' is not defined